In [1]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

In [2]:
class StackedRegressor(BaseEstimator, ClassifierMixin):  
    def __init__(self, classifier=None, regressor=None):
        self.classifier = classifier
        self.regressor = regressor
        
    def fit(self, X, class_labels, regressor_labels):
        self.classifier.fit(X,class_labels)
        self.regressor.fit(X,regressor_labels)
        
    def predict(self, X, y=None):
        class_predict = self.classifier.predict(X)
        regressor_predict = self.regressor.predict(X)
        return np.multiply(class_predict,regressor_predict)

In [3]:
reg = LinearRegression()
classifier = LogisticRegression()

stacked = StackedRegressor(classifier=classifier, regressor=reg)

In [4]:
pd_data = pd.read_csv('../Data/numeric_csv_w_transaction.csv')

In [5]:
X = pd_data.iloc[:,0:31]
regressor_labels = pd_data.iloc[:,-1].fillna(0)


X_train, X_test, regressor_train, regressor_test = train_test_split(
    X, regressor_labels, test_size=0.33, random_state=42)

In [6]:
class_train = (regressor_train!=0).astype(int)
class_test = (regressor_test!=0).astype(int)


In [7]:
stacked.fit(X_train, class_train,regressor_train)

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [8]:
pred = stacked.predict(X_test)

In [11]:
pred.shape

(298206,)